In [6]:
import json
exp_name = 'memory_core_1'
test_data = json.load(open(f'output/{exp_name}/calibrated_cores.json', 'r'))
simple_question_type_examples = json.load(open('data/example/simple_question_type.json', 'r'))

In [7]:
examples = []
for qa in simple_question_type_examples:
    examples.append({'role': 'user', 'content': qa['coreference_resolved_question']})
    examples.append({'role': 'assistant', 'content': qa['simple_question_type']})
examples

[{'role': 'user',
  'content': 'Output: Do Didrik Ficks gränd have Västerlånggatan and Stora Nygatan as terminus?'},
 {'role': 'assistant', 'content': 'verify'},
 {'role': 'user',
  'content': 'Are Canada next to the border of Prince Edward Island and Greenland?'},
 {'role': 'assistant', 'content': 'verify'},
 {'role': 'user',
  'content': 'Is Louise Laroche a civilian of Lindendorf and Wellpond Green?'},
 {'role': 'assistant', 'content': 'verify'},
 {'role': 'user',
  'content': 'How many works of art were exhibited at lesser number of events than Ours blanc ?'},
 {'role': 'assistant', 'content': 'compare_and_count'},
 {'role': 'user',
  'content': 'How many awards are bestowed by greater number of newspapers and German business organizations than Three Physicists Prize?'},
 {'role': 'assistant', 'content': 'compare_and_count'},
 {'role': 'user',
  'content': 'How many political territories have diplomatic relationships with around 44 administrative territories or political territorie

In [8]:
import openai

api_key = 'key'
LLM_engine = 'Qwen/Qwen2.5-32B-Instruct'
openai.base_url = 'http://222.29.156.145:8000/v1/'

# api_key = 'sk-54964e5c3b8c4998a74f7d3e35b618ac'
# LLM_engine = 'deepseek-chat'
# openai.base_url = 'https://api.deepseek.com'

In [9]:
for qa in test_data:
    messages = [{'role': 'system', 'content': """
Question Type Classification Guide (Revised)

Classification Criteria (Strict Priority Order)

1. verify
   Contains verification intent (no implicit assumptions)

2. optimize  
   Contains ANY extreme value keywords:  
   - `max/maximum/most/top`  
   - `min/minimum/least/bottom`
   - `best/worst` (when referring to extremes)

3. compare_and_count  
   Must satisfy ALL:  
   - STARTS WITH "How many"
   - Contains ANY:
     • QUANTITATIVE comparison keywords: `than/more/less/fewer/lesser/at least/at most`
     • Numerical qualifiers: `exactly/around/approximately`
   - Has counting as ultimate goal, which means comparison serves counting

4. compare  
   Must satisfy ALL:  
   - Contains ANY:
     • QUANTITATIVE comparison keywords: `than/more/less/fewer/lesser/at least/at most`
     • Numerical qualifiers: `exactly/around/approximately`
   - NO "How many" opening

5. count
   Must satisfy ALL:
   - STARTS WITH "How many"
   - Lacks ALL: comparison/optimization keywords
                 
6. simple  
   Default type when no above criteria match

Critical Exclusions:
- "but not/other than/excluding" → simple (never compare)

Execution Flow
1. Process input strictly following the priority order  
2. Output only the matched type label, no explanatory text allowed
"""}] + examples + [{'role': 'user', 'content': qa['coreference_resolved_question']}]
    openai.api_key = api_key
    resp = openai.chat.completions.create(
        model=LLM_engine,
        messages=messages,
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    
    qa['predicted_simple_question_type'] = resp.choices[0].message.content

    if qa['coreference_resolved_question'].startswith("How many"):
        if qa['predicted_simple_question_type'] == 'simple':
            qa['predicted_simple_question_type'] = 'count'
        if qa['predicted_simple_question_type'] == 'compare':
            qa['predicted_simple_question_type'] = 'compare_and_count'
    else:
        if qa['predicted_simple_question_type'] == 'compare_and_count':
            qa['predicted_simple_question_type'] = 'compare'
        if qa['predicted_simple_question_type'] == 'count':
            qa['predicted_simple_question_type'] = 'simple'
    
    comparison_keywords = 'than/more/less/fewer/lesser/at least/at most/exactly/around/approximately'.split('/')
    if any([keyword in qa['coreference_resolved_question'] for keyword in comparison_keywords]):
        if qa['predicted_simple_question_type'] == 'simple':
            qa['predicted_simple_question_type'] = 'compare'
        if qa['predicted_simple_question_type'] == 'count':
            qa['predicted_simple_question_type'] = 'compare_and_count'

In [10]:
# for qa in test_data:
#     if qa['predicted_simple_question_type'] != qa['simple_question_type']:
#         print(qa['coreference_resolved_question'], qa['template'], qa['simple_question_type'], qa['predicted_simple_question_type'], sep='\n', end='\n\n')

In [12]:
json.dump(test_data, open(f'output/{exp_name}/predicted_type.json', 'w'), indent=2)